In [ ]:
# Imports
import numpy as np
import pandas as pd
import random
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import statsmodels.api as sm
from statsmodels.tsa.vector_ar.var_model import VAR

import warnings
warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


# Dataset Generation
def generate_multivariate_ts(n_steps=3000, n_features=6, noise_std=0.08):
    t = np.arange(n_steps)
    dates = pd.date_range(start="2000-01-01", periods=n_steps, freq='D')
    data = np.zeros((n_steps, n_features))

    base_trend = 0.0005 * t
    weekly = np.sin(2 * np.pi * t / 7)
    annual = np.sin(2 * np.pi * t / 365.25)

    for i in range(n_features):
        data[:, i] = (
            (i + 1) * 0.2 * base_trend +
            0.4 * weekly * (1 / (i + 1)) +
            0.3 * annual * ((i % 3) + 1)
        )

        ar = np.zeros(n_steps)
        phi = 0.6 - 0.05 * i
        ar[0] = np.random.normal(scale=noise_std)
        for j in range(1, n_steps):
            ar[j] = phi * ar[j - 1] + np.random.normal(scale=noise_std * (1 + 0.1 * i))
        data[:, i] += ar
        data[:, i] += np.random.normal(scale=noise_std, size=n_steps)

    df = pd.DataFrame(data, index=dates, columns=[f"f{i+1}" for i in range(n_features)])
    return df


df = generate_multivariate_ts()
print(df.head())


# Split & Scaling
def train_val_test_split(df, train_frac=0.7, val_frac=0.15):
    n = len(df)
    train = df.iloc[: int(n * train_frac)]
    val = df.iloc[int(n * train_frac): int(n * (train_frac + val_frac))]
    test = df.iloc[int(n * (train_frac + val_frac)):]
    return train, val, test


train_df, val_df, test_df = train_val_test_split(df)

scaler = StandardScaler()
scaler.fit(pd.concat([train_df, val_df]))

train_s = pd.DataFrame(scaler.transform(train_df), index=train_df.index)
val_s = pd.DataFrame(scaler.transform(val_df), index=val_df.index)
test_s = pd.DataFrame(scaler.transform(test_df), index=test_df.index)


# Sequence Creation
def create_sequences(data, input_len=30, horizon=1):
    X, y = [], []
    for i in range(len(data) - input_len - horizon + 1):
        X.append(data[i:i+input_len])
        y.append(data[i+input_len : i+input_len+horizon])
    X, y = np.array(X), np.array(y)
    if horizon == 1:
        y = y.reshape(len(y), -1)
    return X, y


# LSTM Model
def build_lstm(input_len, n_features, units=64, n_layers=1, dropout_rate=0.2, lr=1e-3):
    inp = keras.Input(shape=(input_len, n_features))
    x = inp
    for i in range(n_layers):
        return_seq = (i < n_layers - 1)
        x = layers.LSTM(units, return_sequences=return_seq)(x)
        x = layers.Dropout(dropout_rate)(x)
    out = layers.Dense(n_features)(x)
    model = keras.Model(inp, out)
    model.compile(optimizer=keras.optimizers.Adam(lr), loss="mse", metrics=["mae"])
    return model


# Manual Grid Search
def manual_grid_search(train_scaled):
    results = []
    input_lens = [20, 30]
    units_list = [32, 64]
    dropouts = [0.1, 0.3]

    for il in input_lens:
        X, y = create_sequences(train_scaled.values, il)
        split = int(0.8 * len(X))
        X_tr, X_val = X[:split], X[split:]
        y_tr, y_val = y[:split], y[split:]

        for u in units_list:
            for d in dropouts:
                model = build_lstm(il, train_scaled.shape[1], units=u, dropout_rate=d)
                h = model.fit(X_tr, y_tr, validation_data=(X_val, y_val),
                              epochs=5, batch_size=64, verbose=0)
                val_loss = h.history["val_loss"][-1]
                results.append((il, u, d, val_loss))
    return results


# MC Dropout Sampling
def mc_dropout_predict(model, x, samples=100):
    preds = []
    for _ in range(samples):
        preds.append(model(x, training=True).numpy())
    return np.stack(preds)


# MC Intervals
def mc_intervals(mc_preds, alpha=0.1):
    lower = np.quantile(mc_preds, alpha/2, axis=0)
    upper = np.quantile(mc_preds, 1 - alpha/2, axis=0)
    median = np.median(mc_preds, axis=0)
    return lower, upper, median


# Rolling Forecast
def rolling_forecast(model, history_scaled, input_len, steps_out, samples=100):
    preds, lowers, uppers = [], [], []
    history = history_scaled.copy()

    for _ in range(steps_out):
        window = history[-input_len:]
        x = window.reshape(1, input_len, -1)

        mc = mc_dropout_predict(model, x, samples=samples)
        lower, upper, median = mc_intervals(mc)

        preds.append(median.ravel())
        lowers.append(lower.ravel())
        uppers.append(upper.ravel())

        history = np.vstack([history, median])

    return np.array(preds), np.array(lowers), np.array(uppers)


# VAR Baseline
def var_baseline(train_df, steps):
    model = VAR(train_df)
    res = model.fit(ic="aic")
    p = res.k_ar
    last = train_df.values[-p:]
    forecast = res.forecast(last, steps=steps)
    return forecast


# Metrics
def rmse(a, p): return np.sqrt(mean_squared_error(a, p))
def mae(a, p): return mean_absolute_error(a, p)
def interval_coverage(actual, lower, upper):
    return ((actual >= lower) & (actual <= upper)).mean()


# Pipeline
def run_pipeline(df):
    input_len = 30
    n_features = df.shape[1]

    full_train = pd.concat([train_s, val_s])
    X_train, y_train = create_sequences(full_train.values, input_len)

    model = build_lstm(input_len, n_features, units=64, dropout_rate=0.2)
    model.fit(X_train, y_train, epochs=25, batch_size=64, verbose=1)

    pred_med, lower, upper = rolling_forecast(
        model, full_train.values, input_len, len(test_s), samples=200
    )

    pred_med_un = scaler.inverse_transform(pred_med)
    lower_un = scaler.inverse_transform(lower)
    upper_un = scaler.inverse_transform(upper)
    actual_un = scaler.inverse_transform(test_s)

    metrics = {}
    metrics["LSTM_RMSE"] = rmse(actual_un, pred_med_un)
    metrics["LSTM_MAE"] = mae(actual_un, pred_med_un)
    metrics["LSTM_Interval_Coverage"] = interval_coverage(actual_un, lower_un, upper_un)

    var_fc = var_baseline(train_df, len(test_df))
    metrics["VAR_RMSE"] = rmse(test_df.values, var_fc)
    metrics["VAR_MAE"] = mae(test_df.values, var_fc)

    return model, pred_med_un, lower_un, upper_un, actual_un, var_fc, metrics


# Run
model, pred, lower, upper, actual, var_fc, metrics = run_pipeline(df)

print("\n=== FINAL METRICS ===")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")


# Plot
plt.figure(figsize=(16,6))
plt.plot(actual[:,0], label="Actual (f1)", color="black")
plt.plot(pred[:,0], label="LSTM Prediction (f1)", color="blue")
plt.fill_between(np.arange(len(pred)), lower[:,0], upper[:,0], color="cyan", alpha=0.3)
plt.title("LSTM Forecast + MC Dropout Interval (Feature 1)")
plt.legend()
plt.show()
